In [2]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
	
db = client["lifesnaps"]
collection = db["fitbit"]

person = "621e2f9167b776a240011ccb"

query = {
    "id": ObjectId(person),
    "type": "steps" 
}

doc = collection.find(query)
for doc in collection.find(query):
    print(doc)

In [ ]:
for doc in collection.find(query):
    recorded_time = doc["data"]["dateTime"]
    date = recorded_time.split("T")[0]
    time = recorded_time.split("T")[1]
    steps = doc["data"]["value"]
    print(f"Date: {date}, Time: {time}, Temperature: {steps}")


In [6]:
records = []
pipeline = [
    { "$match": query },
    { "$project": {
        "date": { "$substrBytes": [ "$data.dateTime", 0, 10 ] },
        "valueNum": { "$toInt": "$data.value" }
    }},
    { "$group": {
        "_id": "$date",
        "totalSteps": { "$sum": "$valueNum" }
    }},
    { "$sort": { "_id": 1 } }
]

for doc in collection.aggregate(pipeline):
    #print(f"{doc['_id']}: {doc['totalSteps']} steps")
    records.append({"date": doc["_id"], "steps": doc["totalSteps"]})
    
df = pd.DataFrame(records)

In [ ]:
df

,date,steps
0,2021-10-23,6873
1,2021-10-24,5317
2,2021-10-25,6640
3,2021-10-26,6599
4,2021-10-27,7327
...,...,...
84,2022-01-15,7123
85,2022-01-16,3120
86,2022-01-17,7876
87,2022-01-18,4958


In [ ]:
df_day = df.loc['2021-12-15']

KeyError: '2021-12-15'